In [1]:
%load_ext tensorboard

In [2]:
import numpy as np
import tensorflow as tf
from utils_functions import *
from plan import *
import random
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model
from tensorboard.plugins.hparams import api as hp
import datetime, os

In [3]:
# controllo di star effettivamente usando la GPU

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.config.list_physical_devices('GPU'))

Num GPUs Available:  1
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
dict_stati= load_file("./dizionario_stati")
plansLoaded=load_file("./plans")

File loaded
File loaded


In [32]:
#ogni stato è un tensore con elementi dtype int8, questi stati vengono raggruppati in un altro tensore che rappresnta la
#variabile "states" del piano 

def build_vector(dict, states_list):
    l=len(dict)
    vector_states=[]
    for state in states_list:
        vector=np.array([0]*l,dtype=np.int8)
        #vector=tf.zeros(l,dtype=np.int8)
        for s in state:
            for key in dict.keys():
                if key==s:
                    vector[dict[key]-1]=1
                    break
        #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
        vector_states.append(vector) 
    r = np.array(vector_states)                          
    return r

#NON UTILIZZATO ADESSO
#shape di ogni singolo elemento (r) è (n x 340) con n che varia su ogni piano, raggruppati sulla base dei piani

def build_all_vectors(dict,plans_list):
    total=[]
    for plan in plans_list:
        r=build_vector(dict,plan.states)
        total.append(r)
    #r = tf.convert_to_tensor(total,dtype=None)                         
    return total
 
    
#gli stati vengono ordinati (in una lista) singolarmente con shape (1x340), non vengono raggruppati sulla base dei piani
#da utilizzare per autoencoder standard

def build_all_vectors1x340(dict, plans_list):
    l=len(dict)
    total=[]
    for plan in plans_list:
        for state in plan.states:
            vector=np.array([0]*l,dtype=np.int8)
            for s in state:
                for key in dict.keys():
                    if key==s:
                        vector[dict[key]-1]=1
                        break
            #t=tf.convert_to_tensor(vector,dtype=tf.int8)            
            total.append(vector)
    total=np.array(total)        
    return total        

## Preparazione dati

### Caricamento dataset statici

In [33]:
test=load_file("./Dataset/set_test")
train=load_file("./Dataset/set_training")
validation=load_file("./Dataset/set_validation")

File loaded
File loaded
File loaded


In [34]:
test=np.array(test,dtype=np.int8)
train=np.array(train,dtype=np.int8)
validation=np.array(validation,dtype=np.int8)

In [35]:
logdir = os.path.join("./TestLogs/", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
my_callbacks = [
    tf.keras.callbacks.EarlyStopping(patience=10),
    tf.keras.callbacks.TensorBoard(log_dir=logdir,histogram_freq=1, write_graph=True, write_images=True)
]

In [36]:
# Per il test mi limito a tunare il numero di unità nascoste.
# Se poi funziona, dovrebbe trattarsi solo di scrivere meccanicamente le righe restanti.
input_size = 340
HP_HIDDEN_SIZE = hp.HParam('hidden_size', hp.Discrete([160, 180]))
code_size = 85
# Per il test mi limito ad accuracy
METRIC_ACCURACY = 'accuracy'

with tf.summary.create_file_writer('./logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_HIDDEN_SIZE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

In [40]:
def train_model(hparams):
    input_layer = Input(shape=(input_size,))
    hidden_1 = Dense(hparams[HP_HIDDEN_SIZE], activation='swish', kernel_initializer="he_uniform", )(input_layer)
    code = Dense(code_size, activation='swish',kernel_initializer="he_uniform",)(hidden_1)
    hidden_2 =Dense(hparams[HP_HIDDEN_SIZE], activation='swish',kernel_initializer="he_uniform",)(code)
    output_layer = Dense(input_size, activation='sigmoid')(hidden_2)
    
    autoencoder = Model(input_layer, output_layer)
    autoencoder.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss='binary_crossentropy',
        metrics=['accuracy'],
    )
    autoencoder.fit(
        x=train,y=train,
        epochs=500,
        batch_size=300000,
        validation_data=(validation,validation),
        callbacks=my_callbacks,
    )
#     accuracy = autoencoder.evaluate(test, test)

In [43]:
def run(run_dir, hparams):
    with tf.summary.create_file_writer(run_dir).as_default():
        hp.hparams(hparams)
        accuracy = train_model(hparams)
#         tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [44]:
session_num = 0

for h_units in HP_HIDDEN_SIZE.domain.values:
    hparams= {
        HP_HIDDEN_SIZE: h_units,
    }
    run_name = "run-%d" % session_num
    print('--- Starting trial: %s' % run_name)
    print({h.name: hparams[h] for h in hparams})
    run('logs/hparam_tuning/' + run_name, hparams)
    session_num += 1

--- Starting trial: run-0
{'hidden_size': 160}
Epoch 1/500
4/4 [==============================] - 3s 790ms/step - loss: 0.6905 - accuracy: 0.0054 - val_loss: 0.6771 - val_accuracy: 0.0076
Epoch 2/500
4/4 [==============================] - 1s 148ms/step - loss: 0.6691 - accuracy: 0.0067 - val_loss: 0.6374 - val_accuracy: 0.0040
Epoch 3/500
4/4 [==============================] - 1s 215ms/step - loss: 0.6192 - accuracy: 0.0027 - val_loss: 0.5522 - val_accuracy: 5.1142e-04
Epoch 4/500
4/4 [==============================] - 1s 327ms/step - loss: 0.5186 - accuracy: 2.8280e-04 - val_loss: 0.4094 - val_accuracy: 3.4673e-05
Epoch 5/500
4/4 [==============================] - 1s 318ms/step - loss: 0.3664 - accuracy: 7.5847e-06 - val_loss: 0.2579 - val_accuracy: 0.0000e+00
Epoch 6/500
4/4 [==============================] - 2s 188ms/step - loss: 0.2352 - accuracy: 1.0835e-06 - val_loss: 0.2070 - val_accuracy: 0.0023
Epoch 7/500
4/4 [==============================] - 6s 1s/step - loss: 0.2109 - accu

Epoch 116/500
4/4 [==============================] - 1s 151ms/step - loss: 0.0612 - accuracy: 0.1054 - val_loss: 0.0608 - val_accuracy: 0.1049
Epoch 117/500
4/4 [==============================] - 1s 142ms/step - loss: 0.0606 - accuracy: 0.1044 - val_loss: 0.0602 - val_accuracy: 0.1035
Epoch 118/500
4/4 [==============================] - 1s 145ms/step - loss: 0.0600 - accuracy: 0.1039 - val_loss: 0.0596 - val_accuracy: 0.1038
Epoch 119/500
4/4 [==============================] - 1s 146ms/step - loss: 0.0595 - accuracy: 0.1039 - val_loss: 0.0591 - val_accuracy: 0.1009
Epoch 120/500
4/4 [==============================] - 1s 153ms/step - loss: 0.0590 - accuracy: 0.1024 - val_loss: 0.0586 - val_accuracy: 0.0999
Epoch 121/500
4/4 [==============================] - 1s 388ms/step - loss: 0.0585 - accuracy: 0.1028 - val_loss: 0.0581 - val_accuracy: 0.1034
Epoch 122/500
4/4 [==============================] - 1s 153ms/step - loss: 0.0580 - accuracy: 0.1041 - val_loss: 0.0576 - val_accuracy: 0.1054

4/4 [==============================] - 1s 152ms/step - loss: 0.0328 - accuracy: 0.1396 - val_loss: 0.0327 - val_accuracy: 0.1371
Epoch 231/500
4/4 [==============================] - 1s 209ms/step - loss: 0.0327 - accuracy: 0.1379 - val_loss: 0.0327 - val_accuracy: 0.1405
Epoch 232/500
4/4 [==============================] - 1s 149ms/step - loss: 0.0326 - accuracy: 0.1402 - val_loss: 0.0325 - val_accuracy: 0.1366
Epoch 233/500
4/4 [==============================] - 1s 155ms/step - loss: 0.0325 - accuracy: 0.1375 - val_loss: 0.0324 - val_accuracy: 0.1390
Epoch 234/500
4/4 [==============================] - 1s 149ms/step - loss: 0.0324 - accuracy: 0.1405 - val_loss: 0.0323 - val_accuracy: 0.1348
Epoch 235/500
4/4 [==============================] - 1s 148ms/step - loss: 0.0323 - accuracy: 0.1382 - val_loss: 0.0322 - val_accuracy: 0.1407
Epoch 236/500
4/4 [==============================] - 1s 148ms/step - loss: 0.0322 - accuracy: 0.1410 - val_loss: 0.0321 - val_accuracy: 0.1355
Epoch 237/500

4/4 [==============================] - 1s 284ms/step - loss: 0.0251 - accuracy: 0.1532 - val_loss: 0.0251 - val_accuracy: 0.1608
Epoch 345/500
4/4 [==============================] - 1s 215ms/step - loss: 0.0250 - accuracy: 0.1578 - val_loss: 0.0251 - val_accuracy: 0.1538
Epoch 346/500
4/4 [==============================] - 1s 273ms/step - loss: 0.0250 - accuracy: 0.1565 - val_loss: 0.0250 - val_accuracy: 0.1556
Epoch 347/500
4/4 [==============================] - 2s 380ms/step - loss: 0.0249 - accuracy: 0.1586 - val_loss: 0.0249 - val_accuracy: 0.1556
Epoch 348/500
4/4 [==============================] - 1s 221ms/step - loss: 0.0249 - accuracy: 0.1546 - val_loss: 0.0249 - val_accuracy: 0.1550
Epoch 349/500
4/4 [==============================] - 1s 232ms/step - loss: 0.0249 - accuracy: 0.1562 - val_loss: 0.0249 - val_accuracy: 0.1592
Epoch 350/500
4/4 [==============================] - 1s 222ms/step - loss: 0.0248 - accuracy: 0.1559 - val_loss: 0.0248 - val_accuracy: 0.1578
Epoch 351/500

4/4 [==============================] - 1s 221ms/step - loss: 0.0212 - accuracy: 0.1559 - val_loss: 0.0211 - val_accuracy: 0.1612
Epoch 459/500
4/4 [==============================] - 1s 206ms/step - loss: 0.0211 - accuracy: 0.1631 - val_loss: 0.0212 - val_accuracy: 0.1556
Epoch 460/500
4/4 [==============================] - 1s 209ms/step - loss: 0.0211 - accuracy: 0.1565 - val_loss: 0.0212 - val_accuracy: 0.1619
Epoch 461/500
4/4 [==============================] - 1s 215ms/step - loss: 0.0211 - accuracy: 0.1600 - val_loss: 0.0210 - val_accuracy: 0.1569
Epoch 462/500
4/4 [==============================] - 1s 221ms/step - loss: 0.0210 - accuracy: 0.1572 - val_loss: 0.0210 - val_accuracy: 0.1639
Epoch 463/500
4/4 [==============================] - 1s 219ms/step - loss: 0.0209 - accuracy: 0.1592 - val_loss: 0.0210 - val_accuracy: 0.1629
Epoch 464/500
4/4 [==============================] - 1s 221ms/step - loss: 0.0209 - accuracy: 0.1634 - val_loss: 0.0210 - val_accuracy: 0.1555
Epoch 465/500

Epoch 73/500
4/4 [==============================] - 1s 157ms/step - loss: 0.0982 - accuracy: 0.1092 - val_loss: 0.0971 - val_accuracy: 0.1130
Epoch 74/500
4/4 [==============================] - 1s 151ms/step - loss: 0.0969 - accuracy: 0.1113 - val_loss: 0.0958 - val_accuracy: 0.1135
Epoch 75/500
4/4 [==============================] - 1s 218ms/step - loss: 0.0956 - accuracy: 0.1150 - val_loss: 0.0945 - val_accuracy: 0.1152
Epoch 76/500
4/4 [==============================] - 2s 232ms/step - loss: 0.0943 - accuracy: 0.1137 - val_loss: 0.0932 - val_accuracy: 0.1192
Epoch 77/500
4/4 [==============================] - 1s 273ms/step - loss: 0.0930 - accuracy: 0.1188 - val_loss: 0.0920 - val_accuracy: 0.1193
Epoch 78/500
4/4 [==============================] - 1s 153ms/step - loss: 0.0917 - accuracy: 0.1185 - val_loss: 0.0907 - val_accuracy: 0.1176
Epoch 79/500
4/4 [==============================] - 1s 161ms/step - loss: 0.0905 - accuracy: 0.1190 - val_loss: 0.0895 - val_accuracy: 0.1227
Epoch 

4/4 [==============================] - 1s 152ms/step - loss: 0.0380 - accuracy: 0.1302 - val_loss: 0.0379 - val_accuracy: 0.1306
Epoch 188/500
4/4 [==============================] - 1s 151ms/step - loss: 0.0378 - accuracy: 0.1320 - val_loss: 0.0377 - val_accuracy: 0.1294
Epoch 189/500
4/4 [==============================] - 1s 177ms/step - loss: 0.0376 - accuracy: 0.1293 - val_loss: 0.0376 - val_accuracy: 0.1316
Epoch 190/500
4/4 [==============================] - 1s 170ms/step - loss: 0.0375 - accuracy: 0.1313 - val_loss: 0.0374 - val_accuracy: 0.1296
Epoch 191/500
4/4 [==============================] - 2s 629ms/step - loss: 0.0373 - accuracy: 0.1314 - val_loss: 0.0373 - val_accuracy: 0.1277
Epoch 192/500
4/4 [==============================] - 2s 403ms/step - loss: 0.0372 - accuracy: 0.1300 - val_loss: 0.0372 - val_accuracy: 0.1278
Epoch 193/500
4/4 [==============================] - 1s 154ms/step - loss: 0.0371 - accuracy: 0.1303 - val_loss: 0.0375 - val_accuracy: 0.1289
Epoch 194/500

4/4 [==============================] - 1s 224ms/step - loss: 0.0283 - accuracy: 0.1432 - val_loss: 0.0284 - val_accuracy: 0.1441
Epoch 302/500
4/4 [==============================] - 2s 484ms/step - loss: 0.0283 - accuracy: 0.1459 - val_loss: 0.0284 - val_accuracy: 0.1395
Epoch 303/500
4/4 [==============================] - 1s 176ms/step - loss: 0.0283 - accuracy: 0.1415 - val_loss: 0.0283 - val_accuracy: 0.1480
Epoch 304/500
4/4 [==============================] - 1s 237ms/step - loss: 0.0282 - accuracy: 0.1472 - val_loss: 0.0282 - val_accuracy: 0.1444
Epoch 305/500
4/4 [==============================] - 1s 245ms/step - loss: 0.0281 - accuracy: 0.1451 - val_loss: 0.0282 - val_accuracy: 0.1444
Epoch 306/500
4/4 [==============================] - 1s 159ms/step - loss: 0.0281 - accuracy: 0.1453 - val_loss: 0.0282 - val_accuracy: 0.1452
Epoch 307/500
4/4 [==============================] - 1s 165ms/step - loss: 0.0281 - accuracy: 0.1453 - val_loss: 0.0284 - val_accuracy: 0.1491
Epoch 308/500

4/4 [==============================] - 1s 259ms/step - loss: 0.0240 - accuracy: 0.1612 - val_loss: 0.0240 - val_accuracy: 0.1633
Epoch 416/500
4/4 [==============================] - 1s 208ms/step - loss: 0.0239 - accuracy: 0.1633 - val_loss: 0.0240 - val_accuracy: 0.1583
Epoch 417/500
4/4 [==============================] - 1s 243ms/step - loss: 0.0239 - accuracy: 0.1608 - val_loss: 0.0240 - val_accuracy: 0.1626
Epoch 418/500
4/4 [==============================] - 1s 238ms/step - loss: 0.0239 - accuracy: 0.1623 - val_loss: 0.0240 - val_accuracy: 0.1609
Epoch 419/500
4/4 [==============================] - 1s 189ms/step - loss: 0.0238 - accuracy: 0.1612 - val_loss: 0.0239 - val_accuracy: 0.1613
Epoch 420/500
4/4 [==============================] - 1s 180ms/step - loss: 0.0238 - accuracy: 0.1632 - val_loss: 0.0239 - val_accuracy: 0.1604
Epoch 421/500
4/4 [==============================] - 1s 185ms/step - loss: 0.0238 - accuracy: 0.1612 - val_loss: 0.0239 - val_accuracy: 0.1615
Epoch 422/500

In [45]:
%tensorboard --logdir ./logs/hparam_tuning